# Notebook to explore the data
Simple, rough notebook to take a look at the data and consider how to preprocess it.

In [1]:
import json
import os

# relies on the data being in the ConvFinQA/data/ directory. Please run scripts/unzip_data.sh to unzip the data.
data_dir = os.path.join('..', 'ConvFinQA/data/')

train_data = json.load(open(os.path.join(data_dir, 'train.json')))

## Let's see an example

In [73]:
train_data[500]

{'pre_text': ['through the certegy merger , the company has an obligation to service $ 200 million ( aggregate principal amount ) of unsecured 4.75% ( 4.75 % ) fixed-rate notes due in 2008 .',
  'the notes were recorded in purchase accounting at a discount of $ 5.7 million , which is being amortized over the term of the notes .',
  'the notes accrue interest at a rate of 4.75% ( 4.75 % ) per year , payable semi-annually in arrears on each march 15 and september 15 .',
  'on april 11 , 2005 , fis entered into interest rate swap agreements which have effectively fixed the interest rate at approximately 5.4% ( 5.4 % ) through april 2008 on $ 350 million of the term loan facilities ( or its replacement debt ) and at approximately 5.2% ( 5.2 % ) through april 2007 on an additional $ 350 million of the term loan .',
  'the company has designated these interest rate swaps as cash flow hedges in accordance with sfas no .',
  '133 .',
  'the estimated fair value of the cash flow hedges results 

### Explore the fields

In [3]:
list(train_data[0].keys())

['pre_text',
 'post_text',
 'filename',
 'table_ori',
 'table',
 'qa',
 'id',
 'annotation']

**According to the docs for ConvFinQA: https://github.com/czyssrs/ConvFinQA/blob/main/README.md**

General fields for all data:
```json
"pre_text": "the texts before the table",
"post_text": "the text after the table",
"table": "the table",
"id": "unique example id"
```

The "annotation" field contains the major information for the conversations. If the conversation is the Type I simple conversation, i.e., the decomposition from one FinQA question, then we have the following fields for "annotation" fields:

```json
"annotation": {
  "original_program": "original FinQA question",
  "dialogue_break": "the conversation, as a list of question turns.", 
  "turn_program": "the ground truth program for each question, corresponding to the list in 'dialogue_break'",
  "qa_split": "this field indicates the source of each question turn - 0 if from the decomposition of the first FinQA question, 1 if from the second. For the Type I simple conversations, this field is all 0s.", 
  "exe_ans_list": "the execution results of each question turn."
}
```

To create nice chunks in the vector db, we need to create a func that can combine all the data fields (pre_text, post_text, table) into a single string. Than can be embedded

## Format data

### Metatdata filtering
Metadata filtering is a quick and simple way to improve RAG results, because it's an easy failure point where LLMs can be bad at identifying document categories.

In [83]:
# looking at the JSON data, the filename follows a consistent pattern, so we can at least extract the year of each document
for i in range(0, 3000, 300): # random-ish sample
    print(f"Filename: {train_data[i]['id']} - Year: {train_data[i]['filename'].split('/')[1]} - Question: {train_data[i].get('qa', {}).get('question', 'No question')}")

Filename: Single_JKHY/2009/page_28.pdf-3 - Year: 2009 - Question: what was the percentage change in the net cash from operating activities from 2008 to 2009
Filename: Single_ADI/2011/page_92.pdf-2 - Year: 2011 - Question: what is the net change in unrealized tax benefits during 2010?
Filename: Single_AAP/2012/page_61.pdf-4 - Year: 2012 - Question: how much did the cost of sales change over from 2010 to 2012
Filename: Double_AAPL/2015/page_24.pdf - Year: 2015 - Question: No question
Filename: Single_LMT/2015/page_52.pdf-2 - Year: 2015 - Question: what was the average backlog at year-end in millions from 2013 to 2015?
Filename: Double_AMT/2012/page_121.pdf - Year: 2012 - Question: No question
Filename: Double_SNPS/2012/page_64.pdf - Year: 2012 - Question: No question
Filename: Single_ETR/2004/page_159.pdf-1 - Year: 2004 - Question: what is the growth rate in net revenue in 2004 for entergy arkansas inc.?
Filename: Single_RSG/2014/page_95.pdf-1 - Year: 2014 - Question: what was the percen

Sometimes the documents are written after the year in which they are referring to, e.g:

```Filename: Single_RSG/2014/page_95.pdf-1 - Year: 2014 - Question: what was the percentage decline in the allowance for doubtful accounts in 2013```

So maybe the filter could be a range.

We could do simple NER on the question, extract the year(s) in question, and then only find docs from these years

In [ ]:
# using gliner which is a decent out-of-the-box entity extractor: https://arxiv.org/abs/2311.08526
from gliner import GLiNER

model = GLiNER.from_pretrained("urchade/gliner_medium-v2.1")

In [88]:
text = "what was the average backlog at year-end in millions from 2013 to 2015?"

# we only care about the year
labels = ["Year"]

# Perform entity prediction
entities = model.predict_entities(text, labels, threshold=0.5)

# I think confidence will be high on all questions
print(entities)

for entity in entities:
    print(entity['text'])

[{'start': 58, 'end': 62, 'text': '2013', 'label': 'Year', 'score': 0.9705820679664612}, {'start': 66, 'end': 70, 'text': '2015', 'label': 'Year', 'score': 0.9251722693443298}]
2013
2015


In [97]:
def get_range(question: str) -> tuple:
    """Takes a question, extracts the years mentioned, and takes the latest year.
    Then will create a year containing the latest year and the next year.
    This should encompass a lot of the documts."""

    labels = ["Year"]
    entities = model.predict_entities(question, labels, threshold=0.5)
    
    try:
        # sometimes we get errors if the entraction doesn't convert to an int
        years = (int(entity['text']) for entity in entities)
        max_year = max(years)
        return (max_year + 1, max_year)
    except:
        return ()

print(get_range("what was the average backlog at year-end in millions from 2013 to 2015?"))

(2016, 2015)


In [98]:
for i in range(0, 3000, 21):
    question = train_data[i].get('qa', {}).get('question', None)
    if question is None:
        continue    

    year_range = get_range(question)

    if len(year_range) == 0:
        print(f"No year range found for {question}")
        continue
    
    doc_year = train_data[i]['filename'].split('/')[1]

    if int(doc_year) not in year_range:
        print(f"False: {question} - {doc_year} - {year_range}")
    else:
        print(f"True: {question} - {doc_year} - {year_range}")


True: what was the percentage change in the net cash from operating activities from 2008 to 2009 - 2009 - (2010, 2009)
No year range found for how much higher are the returns of the s&p 500 in the same period ( 2008-2013 ) ? as a percentage .
False: what is the roi of an investment in the o'reilly automotive inc . from 2010 to 2011? - 2015 - (2012, 2011)
False: what is the percentage change in benefits obligations from 2018 to 2019? - 2018 - (2020, 2019)
True: in billions , what would 2018 total operating revenues have been without the mexico business? - 2018 - (2019, 2018)
True: what was the percentage change in the repurchase reserve between 2008 and 2009 , in millions? - 2009 - (2010, 2009)
True: what was the percentage decline in the operating earnings in 2007 of $ 37 million declined from $ 41 - 2007 - (2008, 2007)
True: what are the receivables from the money pool as a percentage of additional common stock dividends paid in 2003? - 2004 - (2004, 2003)
True: what percent higher yi

From the above quick experiment and only looking by eye, it seems like year range _might_ be a useful filter.

In the code I will leave it as a switch, so we can turn it on or off to check results with or without

## Text and table formatting

In [67]:
import pandas as pd

def format_data(chunk: dict) -> dict:
    """Takes a dict from the train.json data, combines the relevant data fields 
    (pre_text, post_text, table) into a single string, and returns a new dict
    
    Args:
        chunk (dict): a dict from the train.json file
    
    Returns:
        a new dict FILL OUT
    """
    
    # TEXT FORMATTING
    # pre text
    pre_text = "\n".join(chunk["pre_text"])

    # looks like all the chunks have the same table structure
    df = pd.DataFrame(chunk["table"][1:],  # Data rows
                        columns=[''] + chunk["table"][0][1:]) 
    table_text = df.to_markdown(
        index=False,
        tablefmt='pipe', 
        numalign='right',
        stralign='left'
    )

    # post text
    post_text = "\n".join(chunk["post_text"])

    # combine text
    total_text = pre_text + "\n\n" + table_text + "\n\n" + post_text

    # GET YEAR
    year = int(chunk["filename"].split('/')[1])
    
    # RETURN DICT
    return {
        "id": chunk["id"],
        "text": total_text,
        "metadata": {"id": chunk["id"], "qa": chunk.get("qa")}, # not all chunks have a qa field
        "year": year
    }


In [70]:
print(format_data(train_data[0])['text'])

26 | 2009 annual report in fiscal 2008 , revenues in the credit union systems and services business segment increased 14% ( 14 % ) from fiscal 2007 .
all revenue components within the segment experienced growth during fiscal 2008 .
license revenue generated the largest dollar growth in revenue as episys ae , our flagship core processing system aimed at larger credit unions , experienced strong sales throughout the year .
support and service revenue , which is the largest component of total revenues for the credit union segment , experienced 34 percent growth in eft support and 10 percent growth in in-house support .
gross profit in this business segment increased $ 9344 in fiscal 2008 compared to fiscal 2007 , due primarily to the increase in license revenue , which carries the highest margins .
liquidity and capital resources we have historically generated positive cash flow from operations and have generally used funds generated from operations and short-term borrowings on our revolv